Gather information about the presence of any other titans. Prepare for war by analyzing them using this machine learning model.

Paste the video path here

In [38]:
video_path = 'BH25/Testing_Data/1.mp4' # video path

These are the preprocessing functions as specified in the training file

In [39]:
import cv2
import numpy as np
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret,frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame,(64,64))
        frames.append(frame)
    cap.release()
    frames = np.array(frames)
    frames = frames/255
    return frames

In [40]:
import cv2
import numpy as np
def preprocess_video_2(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret,frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame,(64,64))
        frames.append(frame)
    cap.release()
    frames = np.array(frames)
    frames = frames/255
    return frames[1]

In [41]:
import cv2 
import numpy as np
def preprocess_video_3(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (64, 64))
        frames.append(frame)
    cap.release()
    frames = np.array(frames)  # Shape: (20, 64, 64, 3)
    return frames.flatten()  # Flatten to 1D array

Importing models and label encoers(for decoding)

In [42]:
import tensorflow as tf
import joblib
from tensorflow.keras.models import load_model
label_encoder = {}
categorical_columns = ['element','motion','power']
for col in categorical_columns:
    label_encoder[col] = joblib.load(f"label_encoder_{col}.joblib")
element_model = load_model('model_element.h5', compile = False)
motion_model = joblib.load('model_categorical_motion.joblib')
power_model = joblib.load('model_categorical_power.joblib')
speed_model = load_model('model_speed.h5', compile = False)
summary_x_model = load_model('model_summary_x2.h5', compile = False)
summary_y_model = load_model('model_summary_y2.h5', compile = False)

In [43]:
# Function to predict the element in the video
def predict_element(video_path):
    inputs = tf.expand_dims(preprocess_video_2(video_path), axis=0)
    predict_array = element_model.predict(inputs)
    predict_element = label_encoder['element'].inverse_transform(tf.argmax(predict_array,axis=1))
    return predict_element[0]

In [44]:
#Predict the motion in the video
prediction_motion = motion_model.predict(tf.expand_dims(preprocess_video_3(video_path),axis=0))
predict_motion = label_encoder['motion'].inverse_transform(prediction_motion)
predict_motion = predict_motion[0]

#Predict the power in the video
prediction_power = power_model.predict(tf.expand_dims(preprocess_video_3(video_path),axis=0))
predict_power = label_encoder['power'].inverse_transform(prediction_power)
predict_power = predict_power[0]

In [ ]:
#Predicting the speed in the video
inputs = tf.expand_dims(preprocess_video(video_path),axis=0)
predict_speed=speed_model.predict(inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step


In [46]:
# For summary prediction
inputs = tf.expand_dims(preprocess_video(video_path),axis=0)
predict_summary_x=summary_x_model.predict(inputs)
predict_summary_y=summary_y_model.predict(inputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step


In [47]:
# Converting the continuous values to numpy arrays and flattening them(converting them to 1D array)
predict_summary_x = np.array(predict_summary_x)
predict_summary_y = np.array(predict_summary_y)
predict_speed = np.array(predict_speed)
predict_summary_x = predict_summary_x.flatten()
predict_summary_y = predict_summary_y.flatten()
predict_speed = predict_speed.flatten()

In [ ]:
# Making the pandas dataframe of the predictions to show it nicely
import pandas as pd
predictions =pd.DataFrame({
    'video_id':video_path,
    'element': predict_element(video_path),  
    'motion': predict_motion,
    'power': predict_power,
    'speed': predict_speed,               
    'video_summary': list(zip(predict_summary_x, predict_summary_y)) 
})

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step


In [49]:
predictions

,video_id,element,motion,power,speed,video_summary
0,BH25/Testing_Data/1.mp4,Boden,zigzag,rot,7.981758,"(0.59568244, -0.16819939)"
